# 4. Predictive Text, Part II: Neural Language Models

N-gram models are an efficient and effective way to make good predictions that are reasonably grammatical. However, they have a key limitation: they can only consider up to *n-1* previous words.

This means, for instance, that in the following input

> First, the woman spread the jelly, and after that ...

An n-gram model might very well suggest *he* as the next word. This is because n-gram models are local models, and cannot keep track of long-distance dependencies. We could pick a much larger *n*, but then our n-grams will be very specific and not match the input very often.

Instead, we can use *neural language models*. Neural models use machine learning and deep neural networks to represent complex relationships between inputs and outputs, and they are the standard today for language modeling and predictive text (as well as basically every common NLP task). The best neural models of today, such as OpenAI's GPT-3, are trained with corpora of *hundreds of billions of words*. Hence, training these sorts of models is completely infeasible for many languages. 

This tutorial will instead focus on neural techniques that work with the limited training corpus we have, while still producing useful results.

In [1]:
import util
import numpy as np
from collections import Counter

corpus = util.load_raw_text(corpus_directory="corpus-eng") # REPLACE WITH YOUR CORPUS FOLDER
corpus = util.preprocess(corpus)
corpus = corpus[:50000] # If your corpus is small, feel free to omit this

print("Corpus length:", len(corpus))

Corpus length: 50000


In [2]:
# Determine the unique vocabulary in the corpus, and then get rid of tokens that appear once
vocabulary = Counter(corpus)
trimmed_vocabulary = [word for word, count in vocabulary.items() if count > 1]
len(trimmed_vocabulary)

2725

In [3]:
# Create the data set

X = []
Y = []

current_sentence = []

for word in corpus:
    if word in [".", "!", "?"]:
        X.append(current_sentence.copy())
        Y.append(current_sentence[1:] + ["</s>"])
        current_sentence = []
    else:
        current_sentence.append(word)

# Add padding tokens to the end of each sequence
max_sentence_len = 100
X_padded = []
Y_padded = []

for x in X:
    if len(x) > max_sentence_len:
        X_padded.append(x[:100])
    else:
        padding_tokens = max_sentence_len - len(x)
        X_padded.append(x + ["<PAD>"] * padding_tokens)
    
for y in Y:
    if len(y) > max_sentence_len:
        Y_padded.append(y[:100])
    else:
        padding_tokens = max_sentence_len - len(y)
        Y_padded.append(y + ["<PAD>"] * padding_tokens)

X_padded = np.array(X_padded)
Y_padded = np.array(Y_padded)
        
print(X_padded[0])
print(Y_padded[0])

['fans' 'gathered' 'today' 'at' 'the' 'los' 'angeles' 'grave' 'of' 'dead'
 'rocker' 'tom' 'paine' 'on' 'the' 'twenty-fourth' 'anniversary' 'of'
 'his' 'death' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>']
['gathered' 'today' 'at' 'the' 'los' 'angeles' 'grave' 'of' 'dead'
 'rocker' 'tom' 'paine' 'on' 'the' 'twenty-fourth' 'anniversary' 'of'
 'his' 'death' '</s>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' 

In [4]:
# Build the model

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input


model = keras.Sequential()
model.add(Input(shape=(None,), dtype="string"))

# Turn text inputs into a list of numbers, with a unique number for each word
string_lookup_layer = layers.StringLookup(mask_token="<PAD>", vocabulary=list(set(corpus)) + ["</s>"])
model.add(string_lookup_layer)

# Add masking layer to skip the pad tokens
model.add(layers.Masking(mask_value = 0, input_shape=(100,)))
                                          
# The embedding layer first converts the number encodings to one-hot encoding
# Then, it produces a dense embedding for each word that includes information about the word's usage
model.add(layers.Embedding(input_dim=string_lookup_layer.vocabulary_size(), output_dim=64))

model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(string_lookup_layer.vocabulary_size(), activation = "softmax")))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Metal device set to: Apple M1 Max


2022-11-01 18:15:51.845954: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-01 18:15:51.846106: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 string_lookup (StringLookup  (None, None)             0         
 )                                                               
                                                                 
 masking (Masking)           (None, None)              0         
                                                                 
 embedding (Embedding)       (None, None, 64)          418560    
                                                                 
 lstm (LSTM)                 (None, None, 64)          33024     
                                                                 
 lstm_1 (LSTM)               (None, None, 64)          33024     
                                                                 
 time_distributed (TimeDistr  (None, None, 6540)       425100    
 ibuted)                                                

In [5]:
from tqdm import tqdm

integer_encoded_Y = np.array([string_lookup_layer(y) for y in tqdm(Y_padded)])
integer_encoded_Y

100%|██████████████████████████████████████| 4011/4011 [00:11<00:00, 338.35it/s]


array([[5211,  372,  289, ...,    0,    0,    0],
       [6125,  398,  238, ...,    0,    0,    0],
       [2328, 6125, 6477, ...,    0,    0,    0],
       ...,
       [3993, 2622, 5745, ...,    0,    0,    0],
       [ 151,  510,  177, ...,    0,    0,    0],
       [1117, 2668, 1482, ...,    0,    0,    0]])

In [6]:
history = model.fit(X_padded, integer_encoded_Y, epochs = 100, verbose = 1)

Epoch 1/100


2022-11-01 18:16:05.253162: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-01 18:16:05.966330: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-01 18:16:06.126263: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-01 18:16:06.300639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-01 18:16:06.547204: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-01 18:16:06.869562: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


126/126 [==============================] - 13s 85ms/step - loss: 3.1661 - accuracy: 0.8715
Epoch 2/100
126/126 [==============================] - 9s 74ms/step - loss: 1.0361 - accuracy: 0.8854
Epoch 3/100
126/126 [==============================] - 9s 73ms/step - loss: 0.9099 - accuracy: 0.8854
Epoch 4/100
126/126 [==============================] - 9s 73ms/step - loss: 0.7786 - accuracy: 0.8877
Epoch 5/100
126/126 [==============================] - 9s 72ms/step - loss: 0.7457 - accuracy: 0.8932
Epoch 6/100
126/126 [==============================] - 9s 71ms/step - loss: 0.7331 - accuracy: 0.8947
Epoch 7/100
126/126 [==============================] - 9s 71ms/step - loss: 0.7247 - accuracy: 0.8957
Epoch 8/100
126/126 [==============================] - 9s 72ms/step - loss: 0.7153 - accuracy: 0.8965
Epoch 9/100
126/126 [==============================] - 9s 71ms/step - loss: 0.7052 - accuracy: 0.8974
Epoch 10/100
126/126 [==============================] - 9s 71ms/step - loss: 0.6957 - accurac

In [7]:
def predict(text: str, n=1):
    text = util.preprocess(text)
    padding_tokens = max_sentence_len - len(text)
    text += ["<PAD>"] * padding_tokens

    predictions = model.predict([text], verbose=True)
    
    # for pred in predictions[0]:
    #     print(string_lookup_layer.get_vocabulary()[np.argmax(pred)])

    # print(predictions.shape)
    prob_dist = predictions[0][-padding_tokens - 1]
    
    
    # choice = string_lookup_layer.get_vocabulary()[np.argmax(prob_dist)]
    
    choice = np.random.choice(string_lookup_layer.get_vocabulary(), n, p=prob_dist)
    return choice

predict("I")

2022-11-01 18:30:58.516108: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-01 18:30:58.574409: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 646ms/step


2022-11-01 18:30:58.757526: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


array(['did'], dtype='<U21')

In [8]:
def generate(prefix: str):
    length = 0
    next_token = ""
    while next_token not in ["</s>", "<PAD>"] and length < 50:
        next_token = predict(prefix)[0]
        prefix += " " + next_token
        length += 1
    return prefix

generate("I am")

1/1 [==============================] - 0s 46ms/step


'I am going to devour when if i came days , i kind of my liver </s>'

In [9]:
import ipywidgets as widgets
from IPython.display import clear_output

text = widgets.Text(
    value='',
    placeholder='Start typing some text...',
    disabled=False
)
out = widgets.Output()
display(text)
display(out)

def on_change(change):
    text = change['new']
    if text[-1] == ' ':
        prediction = predict(text, 3)
        with out:
            clear_output()
            display(" | ".join(prediction))

text.observe(on_change, names=["value"])

Text(value='', placeholder='Start typing some text...')

Output()